In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import pandas as pd
import numpy as np
from time import perf_counter
from sklearn.cluster import SpectralClustering
from sklearn.metrics.cluster import fowlkes_mallows_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
import pickle

In [2]:
raw_npca = np.load('C:\\Users\\Ron Hasson\\Desktop\\ClassWork\\118B\\cogs118b_finalproject\\data\\sketches_raw_nopca.npy')
fc6_npca = np.load('C:\\Users\\Ron Hasson\\Desktop\\ClassWork\\118B\\cogs118b_finalproject\\data\\sketches_fc6_nopca.npy')
raw_pca  = np.load('C:\\Users\\Ron Hasson\\Desktop\\ClassWork\\118B\\cogs118b_finalproject\\data\\sketches_raw_pca.npy')
fc6_pca  = np.load('C:\\Users\\Ron Hasson\\Desktop\\ClassWork\\118B\\cogs118b_finalproject\\data\\sketches_fc6_pca.npy')
metadata = pd.read_csv('C:\\Users\\Ron Hasson\\Desktop\\ClassWork\\118B\\cogs118b_finalproject\\data\\sketches_metadata.csv')

In [3]:
metadata['category_factored'] = LabelEncoder().fit_transform(metadata.category)
metadata_reduced = metadata.loc[metadata['category_factored'].isin([0, 5, 10, 20])]
metadata_reduced.category_factored.value_counts()

0     384
5     384
10    384
20    384
Name: category_factored, dtype: int64

In [4]:
feature_set = [raw_npca, fc6_npca, raw_pca, fc6_pca]
feature_set_reduced = [array[metadata_reduced.iloc[:,i+1].to_numpy()] for i, array in enumerate(feature_set)]

In [5]:
for feature in feature_set_reduced:
    assert feature.shape[0] == 1536

In [6]:
labels_set  = [metadata_reduced.sort_values(col).category_factored.values 
               for col in metadata_reduced.drop(columns=['category', 'category_factored']).columns]

In [7]:
labels_set[1].shape

(1536,)

model = SpectralClustering(n_clusters=4, gamma=1, n_jobs=-1, verbose=True)

start = perf_counter()
model.fit(feature_set_reduced[3])
end = perf_counter()
print("Run time: {}".format(end-start))

score = fowlkes_mallows_score(labels_set[3], model.labels_)
print("Score: {}".format(score))

In [8]:
gammas = [1, 2, 5, 8]
best = (-1, -1)
for i, gamma in enumerate(gammas):
    model = SpectralClustering(n_clusters=4, gamma=gamma, n_jobs=-1)
    model.fit(feature_set_reduced[3].astype(np.float32))
    score = fowlkes_mallows_score(labels_set[3], model.labels_)
    print('Score {}: {}'.format(gamma,score))
    pickle.dump(model, open('./spectral_model_gamma_{}.sav'.format(gamma), 'wb'))

    if score > best[1]:
        best = (i, score)

print('Best Score: index = {}, score = {}'.format(best[0], best[1]))

Score 1: 0.4978816467720553
C:\Users\Ron Hasson\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\core\_methods.py:205: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
C:\Users\Ron Hasson\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\extmath.py:152: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\Ron Hasson\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\extmath.py:152: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\Ron Hasson\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\pairwise.py:314: RuntimeWarning: invalid value encountered in add
  distances += XX
C:\Users\Ron Hasson\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\extmath.py:152: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\Ron Hasson\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\pairwise

In [9]:
scores = []
for i, feature in enumerate(feature_set_reduced):
    model = SpectralClustering(n_clusters=4, gamma=gammas[best[0]], n_jobs=-1).fit(feature_set_reduced[i].astype(np.float32))
    score = fowlkes_mallows_score(labels_set[i], model.labels_)
    print('Score {}: {}'.format(gammas[best[0]], score))
    pickle.dump(model, open('./spectral_model_feature_{}.sav'.format(i), 'wb'))

    scores.append(score)


Score 2: 0.47396337072149597
C:\Users\Ron Hasson\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\core\_methods.py:205: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
C:\Users\Ron Hasson\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\extmath.py:152: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\Ron Hasson\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\pairwise.py:313: RuntimeWarning: overflow encountered in multiply
  distances = - 2 * safe_sparse_dot(X, Y.T, dense_output=True)
C:\Users\Ron Hasson\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\extmath.py:152: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\Ron Hasson\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\pairwise.py:314: RuntimeWarning: invalid value encountered in add
  distances += XX
C:\Users\Ron Hasson\AppData\Local\Programs\Python\P

ValueError: expected 1-d or 2-d array or matrix, got array(None, dtype=object)

In [10]:
scores

[0.47396337072149597, 0.49951116168857107, 0.49951116168857107]

In [11]:
model = SpectralClustering(n_clusters=4, gamma=gammas[best[0]], n_jobs=-1).fit(feature_set_reduced[3].astype(np.float32))
score = fowlkes_mallows_score(labels_set[3], model.labels_)
print('Score {}: {}'.format(gammas[best[0]], score))
pickle.dump(model, open('./spectral_model_feature_{}.sav'.format(3), 'wb'))

scores.append(score)

Score 2: 0.49951116168857107
C:\Users\Ron Hasson\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\core\_methods.py:205: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
C:\Users\Ron Hasson\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\extmath.py:152: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\Ron Hasson\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\pairwise.py:314: RuntimeWarning: invalid value encountered in add
  distances += XX
C:\Users\Ron Hasson\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\extmath.py:152: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\Ron Hasson\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\pairwise.py:314: RuntimeWarning: invalid value encountered in add
  distances += XX
C:\Users\Ron Hasson\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\extma

In [13]:
scores

[0.47396337072149597,
 0.49951116168857107,
 0.49951116168857107,
 0.49951116168857107]

In [14]:
scores

[0.47396337072149597,
 0.49951116168857107,
 0.49951116168857107,
 0.49951116168857107]